In [ ]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree

# Para realizar la clasificación y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, learning_curve, GridSearchCV, cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    cohen_kappa_score,
    confusion_matrix
)
import xgboost as xgb
import pickle
import shap
import time
import psutil

# Para realizar cross validation
# -----------------------------------------------------------------------
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from sklearn.preprocessing import KBinsDiscretizer


import warnings  
warnings.filterwarnings("ignore") 

In [ ]:
import sys
sys.path.append("../")

from src.support_models import ClassificationModels

In [ ]:
df = pd.read_csv('../data/output/complete_data_preprocessed.csv', index_col=0)

In [ ]:
df.head()

### Logistic regression

In [ ]:
models = ClassificationModels(df, "Attrition")

In [ ]:
model_list = ["logistic_regression", "tree", "random_forest", "gradient_boosting", "xgboost"]

for model in model_list:

    models.fit_model(model)
    models.plot_confusion_matrix(model)
    models.plot_predictors_importance(model)
    models.plot_shap_summary(model)
    df_results = models.get_metrics(model)

In [ ]:

# llamamos a la función para ajustar el modelo
logistic.ajustar_modelo("logistic_regression")

logistic.plot_matriz_confusion("logistic_regression")
logistic.importancia_predictores(modelo_nombre ='logistic_regression')
logistic.plot_shap_summary(modelo_nombre="logistic_regression")

# llamamos al metodo para obtener los resultaods de las predicciones
df_resultados = logistic.calcular_metricas(modelo_nombre =  "logistic_regression" )
df_resultados["modelo"] = "regresion logistica"
df_resultados